# Custom Named Entity Recognition- Student Resume

### Load Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm.notebook import tqdm
import json
import random
import logging
import re

In [3]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 31.6 MB/s 
     |████████████████████████████████| 596 kB 68.8 MB/s 
     |████████████████████████████████| 895 kB 72.8 MB/s 
     |████████████████████████████████| 3.3 MB 60.6 MB/s 
     |████████████████████████████████| 67 kB 6.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
from transformers import AutoTokenizer, TFAutoModelForTokenClassification

### Load and Preprocess data


In [5]:
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    try:
        training_data = []
        lines=[]
        with open(dataturks_JSON_FilePath, 'r') as f:
            lines = f.readlines()
        for line in lines:
            data = json.loads(line)
            text = data['content'].replace("\n", " ")
            entities = []
            data_annotations = data['annotation']
            if data_annotations is not None:
                for annotation in data_annotations:
                    point = annotation['points'][0]
                    labels = annotation['label']
                    if not isinstance(labels, list):
                        labels = [labels]
                    for label in labels:
                        point_start = point['start']
                        point_end = point['end']
                        point_text = point['text']
                        lstrip_diff = len(point_text) - len(point_text.lstrip())
                        rstrip_diff = len(point_text) - len(point_text.rstrip())
                        if lstrip_diff != 0:
                            point_start = point_start + lstrip_diff
                        if rstrip_diff != 0:
                            point_end = point_end - rstrip_diff
                        entities.append((point_start, point_end + 1 , label))
            training_data.append((text, {"entities" : entities}))
        return training_data
    except Exception as e:
        logging.exception("Unable to process " + dataturks_JSON_FilePath + "\n" + "error = " + str(e))
        return None

In [6]:
def trim_entity_spans(data: list) -> list:
    invalid_span_tokens = re.compile(r'\s')
    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])
    return cleaned_data  

In [7]:
def clean_dataset(data):
    cleanedDF = pd.DataFrame(columns=["setences_cleaned"])
    sum1 = 0
    for i in tqdm(range(len(data))):
        start = 0
        emptyList = ["Empty"] * len(data[i][0].split())
        numberOfWords = 0
        lenOfString = len(data[i][0])
        strData = data[i][0]
        strDictData = data[i][1]
        lastIndexOfSpace = strData.rfind(' ')
        for i in range(lenOfString):
            if (strData[i]==" " and strData[i+1]!=" "):
                for k,v in strDictData.items():
                    for j in range(len(v)):
                        entList = v[len(v)-j-1]
                        if (start>=int(entList[0]) and i<=int(entList[1])):
                            emptyList[numberOfWords] = entList[2]
                            break
                        else:
                            continue
                start = i + 1  
                numberOfWords += 1
            if (i == lastIndexOfSpace):
                for j in range(len(v)):
                        entList = v[len(v)-j-1]
                        if (lastIndexOfSpace>=int(entList[0]) and lenOfString<=int(entList[1])):
                            emptyList[numberOfWords] = entList[2]
                            numberOfWords += 1
        cleanedDF = cleanedDF.append(pd.Series([emptyList],  index=cleanedDF.columns ), ignore_index=True )
        sum1 = sum1 + numberOfWords
    return cleanedDF

In [8]:
def tag_dicts(cleanedDF):
  unique_tags = set(cleanedDF['setences_cleaned'].explode().unique())
  tag2id = {tag: id for id, tag in enumerate(unique_tags)}
  id2tag = {id: tag for tag, id in tag2id.items()}
  return tag2id, id2tag, unique_tags

In [9]:
def tagging_and_padding(cleanedDF, tag2id):
  MAX_LEN = 512
  labels = cleanedDF['setences_cleaned'].values.tolist()
  tags = pad_sequences([[tag2id.get(l) for l in lab] for lab in labels], maxlen=MAX_LEN, value=tag2id["Empty"], padding="post", dtype="long", truncating="post")
  return tags

In [10]:
def load_and_prepare_data(filepath):
  df_data = pd.read_json(filepath, lines=True)
  df_data = df_data.drop(['extras'], axis=1)
  df_data['content'] = df_data['content'].str.replace("\n", " ")
  data = convert_dataturks_to_spacy(filepath)
  data = trim_entity_spans(data)
  cleanedDF= clean_dataset(data)
  tag2id, id2tag, unique_tags= tag_dicts(cleanedDF)
  tags= tagging_and_padding(cleanedDF, tag2id)
  return df_data['content'].values.tolist(), tags, unique_tags

### Tokenize and ready the data

In [11]:
def tokenize_and_align_labels(tokenizer, examples, tags):
    tokenized_inputs = tokenizer(examples, truncation=True, is_split_into_words=False, padding='max_length', max_length=512)
    labels = []
    for i, label in enumerate(tags):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

### Train model

In [12]:
filepath= "/content/drive/MyDrive/NER_Resume/Entity Recognition in Resumes.json"

In [13]:
df, tags, unique_tags= load_and_prepare_data(filepath)

  0%|          | 0/220 [00:00<?, ?it/s]

In [14]:
label_all_tokens = True
model_type= "distilbert-base-uncased"

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_type)
test = tokenize_and_align_labels(tokenizer, df, tags)
train_dataset = tf.data.Dataset.from_tensor_slices((test['input_ids'], test['labels']))

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [16]:
model = TFAutoModelForTokenClassification.from_pretrained(model_type,num_labels=len(unique_tags))

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForTokenClassification: ['vocab_projector', 'activation_13', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inferenc

In [18]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
model.fit(train_dataset.shuffle(1000).batch(16), epochs=3, batch_size=16)

Epoch 1/3


/usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:377: UserWarning: Using `-1` to mask the loss for the token is deprecated. Please use `-100` instead.
  return py_builtins.overload_of(f)(*args)


14/14 [==============================] - 19s 840ms/step - loss: 1.2233 - accuracy: 0.7476
Epoch 2/3
14/14 [==============================] - 12s 847ms/step - loss: 0.5657 - accuracy: 0.7537
Epoch 3/3
14/14 [==============================] - 12s 855ms/step - loss: 0.5094 - accuracy: 0.7537
